# Max Cut QAOA - Pennylane

In this notebook, we present an implementation of the Hybrid Max Cut QAOA using Xanadu's Pennylane Python framework. [Here](https://pennylane.ai/qml/demos/tutorial_qaoa_maxcut.html) is the source.

For information about the max cut problem, see [here](https://en.wikipedia.org/wiki/Maximum_cut).

The first step is to import the Pennylane package and set up our graph. Make sure you have Pennylane installed:
```terminal
pip install pennylane
```

In [1]:
import pennylane as qml
from pennylane import numpy as np

n_vertices = 4
graph = [(0, 1), (0, 3), (1, 2), (2, 3)]

As you can see, we specify the number of vertices and define the edge list of the graph. Visually, our graph looks like this:

![Graph in question](graph.png)

Our QAOA will be made up of __2p__ parameters and layers. Each layer will be defined by a parameterizable unitary operator. The cost layers will be comprised of the operator U_C and are parametrized by gamma_1, ..., gamma_p and similarly the mixing layers will be made up of U_B paramterized by beta_1, ..., beta_p. We define these custom gates below:

In [2]:
n_qubits = n_vertices

# Mixing operator, a one qubit gate
def U_B(beta_i):
    for qubit in range(n_qubits):
        qml.RX(2 * beta_i, wires = qubit)

# The cost operator is a two qubit gate, as it contains a few CNOT's
def U_C(gamma_i):
    for edge in graph:
        qubit1 = edge[0]
        qubit2 = edge[1]
        qml.CNOT(wires = [qubit1, qubit2])
        qml.RZ(gamma_i, wires = qubit2)
        qml.CNOT(wires = [qubit1, qubit2])

With these custom unitary's in hand, we can continue on to define the quantum circuit and specify the quantum backend it will run on. We put all of the qubits in a uniform superposition with Hadamard gates, perform the cost and mixing steps __p__ times, and then measure some subset of qubits (or __wires__, in the Pennylane terminology).

In [3]:
# Backend
device = qml.device("default.qubit", wires = n_qubits, shots = 1)

# Used to measure in the computational basis
pauli_z = [[1, 0], [0, -1]]
pauli_z_2 = np.kron(pauli_z, pauli_z, requires_grad = False)

# Marks this function as a "QNode", meaning it should be run on quantum hardware
@qml.qnode(device)
def circuit(gammas, betas, edge=None, p=1):
    for qubit in range(n_qubits):
        qml.Hadamard(wires = qubit)
    
    for i in range(p):
        U_C(gammas[i])
        U_B(betas[i])

    if edge is None:
        return qml.sample()

    return qml.expval(qml.Hermitian(pauli_z_2, wires = edge))

Finally, we define a utility function for converting between bit strings and base 10 integers. Now, we're ready to run our QAOA. This takes place in several steps:

1. Initialize parameters
2. Define the "objective function" we wish the classical optimizer to minimize (or maximize the negative)

In [10]:
def bitstring_to_int(bit_string_sample):
    bit_string = "".join(str(bs) for bs in bit_string_sample)
    return int(bit_string, base=2)

def qaoa_maxcut(p=1):
    print("\np={:d}".format(p))

    # initialize the parameters near zero
    init_params = 0.01 * np.random.rand(2, p, requires_grad=True)
    # print(init_params)
    # init_params = np.tensor_wrapper([1.0, 1.0], requires_grad=True)

    # minimize the negative of the objective function
    def objective(params):
        gammas = params[0]
        betas = params[1]
        neg_obj = 0
        for edge in graph:
            # objective for the MaxCut problem
            neg_obj -= 0.5 * (1 - circuit(gammas, betas, edge=edge, p=p))
        return neg_obj

    # initialize optimizer: Adagrad works well empirically
    opt = qml.AdagradOptimizer(stepsize=0.5)

    # optimize parameters in objective
    params = init_params
    steps = 1
    for i in range(steps):
        params = opt.step(objective, params)
        # if (i + 1) % 5 == 0:
        print("Objective after step {:5d}: {: .7f}".format(i + 1, -objective(params)))

    # sample measured bitstrings 100 times
    bit_strings = []
    n_samples = 100
    for i in range(0, n_samples):
        bit_strings.append(bitstring_to_int(circuit(params[0], params[1], edge=None, p=p)))

    # print optimal parameters and most frequently sampled bitstring
    counts = np.bincount(np.array(bit_strings))
    most_freq_bit_string = np.argmax(counts)
    print("Optimized (gamma, beta) vectors:\n{}".format(params[:, :p]))
    print("Most frequently sampled bit string is: {:04b}".format(most_freq_bit_string))

    return -objective(params), bit_strings


# perform qaoa on our graph with p=1,2 and
# keep the bitstring sample lists
bitstrings1 = qaoa_maxcut(p=1)[1]
bitstrings2 = qaoa_maxcut(p=2)[1]

# Transpilation
# Dump to Qiskit -> QASM


p=1
Objective after step     1:  1.0000000
Optimized (gamma, beta) vectors:
[[0.50409429]
 [0.50612332]]
Most frequently sampled bit string is: 0000

p=2
Objective after step     1:  3.0000000
Optimized (gamma, beta) vectors:
[[ 0.50212234 -0.49052642]
 [-0.49847768  0.50738513]]
Most frequently sampled bit string is: 1010
